In [ ]:
import ROOT

pen the rootfile and get the workspace from the exercise_0

In [ ]:
fInput = ROOT.TFile("Workspace_mumufit.root")
ws = fInput.Get("ws")
ws.Print()

ou can set constant parameters that are known<br>
f you leave them floating, the fit procedure will determine their uncertainty

In [ ]:
ws.var("meanJpsi").setConstant(1)
ws.var("sigmaJpsi").setConstant(1)
ws.var("alphaJpsi").setConstant(1)
ws.var("nJpsi").setConstant(1)
ws.var("NJpsi").setConstant(1)
ws.var("meanpsi2S").setConstant(1)
ws.var("Nbkg").setConstant(1)
ws.var("a1").setConstant(1)
ws.var("a2").setConstant(1)
ws.var("a3").setConstant(1)

et the model know what is the parameter of interest

In [ ]:
cross_psi = ws.var("cross_psi")
cross_psi.setRange(4., 16.)  #this is mostly for plotting reasons
poi = ROOT.RooArgSet(cross_psi)

onfigure the model

In [ ]:
model = ROOT.RooStats.ModelConfig()
model.SetWorkspace(ws)
model.SetPdf("totPDF")
model.SetParametersOfInterest(poi)

et confidence level

In [ ]:
confidenceLevel = 0.68

uild the profile likelihood calculator

In [ ]:
plc = ROOT.RooStats.ProfileLikelihoodCalculator(ws.data("data"), model)
plc.SetParameters(poi)
plc.SetConfidenceLevel(confidenceLevel)

et the interval

In [ ]:
pl_Interval = plc.GetInterval()

ow let's determine the Bayesian probability interval<br>
e could use the standard Bayesian Calculator, but this would be very slow for the integration<br>
o we profit of the Markov-Chain MC capabilities of RooStats to speed things up

In [ ]:
mcmc = ROOT.RooStats.MCMCCalculator(ws.data("data") , model)
mcmc.SetConfidenceLevel(confidenceLevel)
mcmc.SetNumIters(20000)           #Metropolis-Hastings algorithm iterations
mcmc.SetNumBurnInSteps(100)       #first N steps to be ignored as burn-in
mcmc.SetLeftSideTailFraction(0.5) #for central interval

In [ ]:
MCMC_interval = mcmc.GetInterval()

et's make a plot

In [ ]:
dataCanvas = ROOT.TCanvas("dataCanvas")
dataCanvas.Divide(2,1)

In [ ]:
dataCanvas.cd(1)
plot_Interval = ROOT.RooStats.LikelihoodIntervalPlot(pl_Interval)
plot_Interval.SetTitle("Profile Likelihood Ratio")
plot_Interval.SetMaximum(3.)
plot_Interval.Draw()

In [ ]:
dataCanvas.cd(2)
plot_MCMC = ROOT.RooStats.MCMCIntervalPlot(MCMC_interval)
plot_MCMC.SetTitle("Bayesian probability interval (Markov Chain)")
plot_MCMC.Draw()

In [ ]:
dataCanvas.SaveAs("exercise_4.png")

ow print the interval for mH for the two methods

In [ ]:
print "PLC interval is [", pl_Interval.LowerLimit(cross_psi), ", ", pl_Interval.UpperLimit(cross_psi), "]"

In [ ]:
print "Bayesian interval is [", MCMC_interval.LowerLimit(cross_psi), ", ", MCMC_interval.UpperLimit(cross_psi), "]"

yROOT sometimes fails cleaning memory, this helps

In [ ]:
del plc